In [98]:
import sys
sys.path.append('~/ner')
import config
from tqdm.notebook import tqdm
import csv

In [99]:
cursor = config.coll.aggregate([{
    '$match': {
        '$and': [{
            'primary_outcomes': {'$not': {'$size': 0}}},{
            'secondary_outcomes': {'$not': {'$size': 0}}}
]}}, 
    {
        '$sample': {'size': 1000}},
    {
        '$project':{'primary_outcomes':1, 'secondary_outcomes':1, 'conditions': 1}}])


In [100]:
class Rule_based_annotation():
    def __init__(self, cursor):
        self.cursor = cursor
    
    def expand_outcomes(self, measure, item):
        self.tag_corpus['NCT'].append('%s; %s'%(item['_id'], ', '.join(item['conditions'])))
        self.tag_corpus['measure'].append(measure['measure'])
        
        get_val = lambda x: measure[x] if measure.get(x)!=None else ''
        
        self.tag_corpus['safety_issue'].append(get_val('safety_issue'))
        self.tag_corpus['description'].append(get_val('description'))
    
    def parse_cursor(self):
        self.tag_corpus = {'NCT': [],'measure': [], 'safety_issue': [], 'description': [], 'time_frame': []}
        for item in self.cursor:
            for measure in item['primary_outcomes']: 
                 self.expand_outcomes(measure, item)
            for measure in item['secondary_outcomes']:
                 self.expand_outcomes(measure, item)

In [101]:
cls = Rule_based_annotation(cursor)

In [102]:
cls.parse_cursor()

In [116]:
cls.tag_corpus['NCT']

['NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT02551094; Coronary Artery Disease, Myocardial Infarction',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT01840904; Acute Diverticulitis',
 'NCT02137941; Stress, Anxiety',
 'NCT02137941; Stress, Anxiety',
 'NC

In [115]:
cls.tag_corpus['measure']

['Time from randomization to the first event of cardiovascular death, resuscitated cardiac arrest, acute myocardial infarction, stroke, or urgent hospitalization for angina requiring coronary revascularization',
 'Time from randomization to death (total mortality)',
 'Time from randomization to cardiovascular death',
 'Time from randomization to resuscitated cardiac arrest',
 'Time from randomization to myocardial infarction',
 'Time from randomization to stroke',
 'Time from randomization to urgent hospitalization for angina requiring coronary revascularization',
 'Time from randomization to the first event of cardiovascular death, resuscitated cardiac arrest, acute MI or stroke.',
 'Changes in Sleep quality during acute uncomplicated diverticulitis from baseline score at remission (14th, 21st and 30th day)',
 'Changes in Sleepiness during acute uncomplicated diverticulitis from baseline score at remission (14th, 21st and 30th day)',
 'Changes in fatigue during acute uncomplicated div

In [130]:
cls.tag_corpus['measure'][70:78]

['Knee Injury an Osteoarhtritis Outcome Score (KOOS)',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Change in Musculoskeletal Symptom Sub-scale on the Breast Cancer Prevention Trial Symptom Scale']

In [131]:

cls.tag_corpus['measure'][70:78]

['Knee Injury an Osteoarhtritis Outcome Score (KOOS)',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Bone microarchitecture of the distal radius',
 'Change in Musculoskeletal Symptom Sub-scale on the Breast Cancer Prevention Trial Symptom Scale']

In [82]:
with open('./data/tag_100.txt', 'w') as f:
    w = csv.writer(f,delimiter = '\n')
    w.writerow(tag_corpus)

In [132]:
cls.tag_corpus['description'][70:78]

['Patient reported symptoms, stiffness, pain, activities of daily living (ADL), function in sport and recreation and knee related quality of life',
 'trabecular bone mineral density measurement (XCT)',
 'bone volume fraction (%)',
 'trabecular homogeneity (mm)',
 'trabecular number (mm-1)',
 'trabecular thickness (mm)',
 'trabecular separation (mm)',
 'The MS subscale is a self-reported measure on a scale of 0 to 4, with lower score indicating less arthralgia/myalgia']

In [129]:
cls.tag_corpus['NCT'][33:40]

['NCT02788461; Non-Small Cell Lung Cancer',
 'NCT02788461; Non-Small Cell Lung Cancer',
 'NCT02788461; Non-Small Cell Lung Cancer',
 "NCT01569503; Crohn's Disease",
 "NCT01569503; Crohn's Disease",
 "NCT01569503; Crohn's Disease",
 "NCT01569503; Crohn's Disease"]